# TuringBench

**Models**
- Bert
- RoBERTa
- GPT-2
- XLNet
- Grover
- XLM
- CTRL

This notebook does AI-generated Text Detection with different NLP models. In a proposed method, adversarial training is applied through these models. The aim is to enhance the original models' performances by utilizing this method through the training phase.

In [ ]:
#!pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import torch
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import AdamW, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from transformers import BertForSequenceClassification, BertTokenizer
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import XLMTokenizer, XLMForSequenceClassification
from transformers import CTRLTokenizer, CTRLForSequenceClassification

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('reviews.csv')
data.head()

In [ ]:
# Define a custom dataset class
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Prepare the dataset for training
texts = data['review'].tolist()
labels = data['label'].apply(lambda x: 1 if x == 'ai' else 0).tolist()

In [ ]:
# Split data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

## Models

### BERT

In [ ]:
# Initialize the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Prepare the data for BERT
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

In [ ]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=1,              # Number of training epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
    evaluation_strategy='steps',     # Evaluation strategy to adopt during training
    eval_steps=100,                  # Evaluation step to perform evaluation
    save_steps=100,                  # Save checkpoint every X steps
    log_level='info',                # Set logging level to info
    log_level_replica='info'         # Save checkpoint every X steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

In [ ]:
# Evaluate the model
bert_results = trainer.evaluate()
print(bert_results)

In [ ]:
# Perform prediction on the test dataset
predictions = trainer.predict(test_dataset)
preds = predictions.predictions
pred_labels = np.argmax(preds, axis=1)  # Get the predicted labels

# If you need the true labels
true_labels = [example['labels'] for example in test_dataset]

# Plot confusion matrix
conf_matrix = confusion_matrix(true_labels, pred_labels)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix for Test Set')
plt.show()

# Print classification report
report = classification_report(true_labels, pred_labels, target_names=['Human', 'AI'])
print(report)

In [ ]:
model_save_path = './test/bert_model'
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

### RoBERTa

In [ ]:
# Initialize tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Tokenization
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results_roberta',  # Output directory
    num_train_epochs=1,              # Number of training epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs_roberta',    # Directory for storing logs
    logging_steps=10,
    evaluation_strategy='steps',     # Evaluation strategy to adopt during training
    eval_steps=100,                  # Evaluation step to perform evaluation
    save_steps=100,                  # Save checkpoint every X steps
    log_level='info',                # Set logging level to info
    log_level_replica='info'         # Save checkpoint every X steps
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

In [ ]:
# Evaluate the model
roberta_results = trainer.evaluate()
print(roberta_results)

In [ ]:
# Perform prediction on the test dataset
predictions = trainer.predict(test_dataset)
preds = predictions.predictions
pred_labels = np.argmax(preds, axis=1)  # Get the predicted labels

# If you need the true labels
true_labels = [example['labels'] for example in test_dataset]

# Plot confusion matrix
conf_matrix = confusion_matrix(true_labels, pred_labels)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix for Test Set')
plt.show()

# Print classification report
report = classification_report(true_labels, pred_labels, target_names=['Human', 'AI'])
print(report)

In [ ]:
model_save_path = './test/roberta_model'
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

### GPT-2

In [ ]:
# Initialize the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)

# Prepare the data for GPT-2
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

In [ ]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results_gpt2',     # Output directory
    num_train_epochs=1,              # Number of training epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
    evaluation_strategy='steps',     # Evaluation strategy to adopt during training
    eval_steps=100,                  # Evaluation step to perform evaluation
    save_steps=100,                  # Save checkpoint every X steps
    log_level='info',                # Set logging level to info
    log_level_replica='info'         # Save checkpoint every X steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

In [ ]:
# Evaluate the model
gpt2_results = trainer.evaluate()
print(gpt2_results)

In [ ]:
# Perform prediction on the test dataset
predictions = trainer.predict(test_dataset)
preds = predictions.predictions
pred_labels = np.argmax(preds, axis=1)  # Get the predicted labels

# If you need the true labels
true_labels = [example['labels'] for example in test_dataset]

# Plot confusion matrix
conf_matrix = confusion_matrix(true_labels, pred_labels)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix for Test Set')
plt.show()

# Print classification report
report = classification_report(true_labels, pred_labels, target_names=['Human', 'AI'])
print(report)

In [ ]:
model_save_path = './test/gpt2_model'
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

### XLNet

In [ ]:
# Initialize the tokenizer and model
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)

# Prepare the data for XLNet
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

In [ ]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results_xlnet',    # Output directory
    num_train_epochs=1,              # Number of training epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs_xlnet',      # Directory for storing logs
    logging_steps=10,
    evaluation_strategy='steps',     # Evaluation strategy to adopt during training
    eval_steps=100,                  # Evaluation step to perform evaluation
    save_steps=100,                  # Save checkpoint every X steps
    log_level='info',                # Set logging level to info
    log_level_replica='info'         # Save checkpoint every X steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(results)

In [ ]:
# Perform prediction on the test dataset
predictions = trainer.predict(test_dataset)
preds = predictions.predictions
pred_labels = np.argmax(preds, axis=1)  # Get the predicted labels

# If you need the true labels
true_labels = [example['labels'] for example in test_dataset]

# Plot confusion matrix
conf_matrix = confusion_matrix(true_labels, pred_labels)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix for Test Set')
plt.show()

# Print classification report
report = classification_report(true_labels, pred_labels, target_names=['Human', 'AI'])
print(report)

In [ ]:
model_save_path = './test/xlnet_model'
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

### Grover

In [ ]:
# Initialize the tokenizer and model from the Grover model
tokenizer = AutoTokenizer.from_pretrained('grover-base')
model = AutoModelForSequenceClassification.from_pretrained('grover-base', num_labels=2)

# Prepare the data for Grover
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

In [ ]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results_grover',    # Output directory
    num_train_epochs=1,              # Number of training epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs_grover',      # Directory for storing logs
    logging_steps=10,
    evaluation_strategy='steps',     # Evaluation strategy to adopt during training
    eval_steps=100,                  # Evaluation step to perform evaluation
    save_steps=100,                  # Save checkpoint every X steps
    log_level='info',                # Set logging level to info
    log_level_replica='info'         # Save checkpoint every X steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

In [ ]:
# Evaluate the model
grover_results = trainer.evaluate()
print(grover_results)

In [ ]:
# Perform prediction on the test dataset
predictions = trainer.predict(test_dataset)
preds = predictions.predictions
pred_labels = np.argmax(preds, axis=1)  # Get the predicted labels

# If you need the true labels
true_labels = [example['labels'] for example in test_dataset]

# Plot confusion matrix
conf_matrix = confusion_matrix(true_labels, pred_labels)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix for Test Set')
plt.show()

# Print classification report
report = classification_report(true_labels, pred_labels, target_names=['Human', 'AI'])
print(report)

In [ ]:
model_save_path = './test/grover_model'
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

### XLM

In [ ]:
# Initialize the tokenizer and model
tokenizer = XLMTokenizer.from_pretrained('xlm-mlm-en-2048')
model = XLMForSequenceClassification.from_pretrained('xlm-mlm-en-2048', num_labels=2)

# Prepare the data for XLM
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

In [ ]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results_xlm',    # Output directory
    num_train_epochs=1,              # Number of training epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs_xlm',      # Directory for storing logs
    logging_steps=10,
    evaluation_strategy='steps',     # Evaluation strategy to adopt during training
    eval_steps=100,                  # Evaluation step to perform evaluation
    save_steps=100,                  # Save checkpoint every X steps
    log_level='info',                # Set logging level to info
    log_level_replica='info'         # Save checkpoint every X steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

In [ ]:
# Evaluate the model
xlm_results = trainer.evaluate()
print(xlm_results)

In [ ]:
# Perform prediction on the test dataset
predictions = trainer.predict(test_dataset)
preds = predictions.predictions
pred_labels = np.argmax(preds, axis=1)  # Get the predicted labels

# If you need the true labels
true_labels = [example['labels'] for example in test_dataset]

# Plot confusion matrix
conf_matrix = confusion_matrix(true_labels, pred_labels)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix for Test Set')
plt.show()

# Print classification report
report = classification_report(true_labels, pred_labels, target_names=['Human', 'AI'])
print(report)

In [ ]:
model_save_path = './test/xlm_model'
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

### CTRL

In [ ]:
# Initialize the tokenizer and model
tokenizer = CTRLTokenizer.from_pretrained('ctrl')
model = CTRLForSequenceClassification.from_pretrained('ctrl', num_labels=2)

# Prepare the data for CTRL
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

In [ ]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results_ctrl',    # Output directory
    num_train_epochs=1,              # Number of training epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs_ctrl',      # Directory for storing logs
    logging_steps=10,
    evaluation_strategy='steps',     # Evaluation strategy to adopt during training
    eval_steps=100,                  # Evaluation step to perform evaluation
    save_steps=100,                  # Save checkpoint every X steps
    log_level='info',                # Set logging level to info
    log_level_replica='info'         # Save checkpoint every X steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

In [ ]:
# Evaluate the model
ctrl_results = trainer.evaluate()
print(ctrl_results)

In [ ]:
# Perform prediction on the test dataset
predictions = trainer.predict(test_dataset)
preds = predictions.predictions
pred_labels = np.argmax(preds, axis=1)  # Get the predicted labels

# If you need the true labels
true_labels = [example['labels'] for example in test_dataset]

# Plot confusion matrix
conf_matrix = confusion_matrix(true_labels, pred_labels)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix for Test Set')
plt.show()

# Print classification report
report = classification_report(true_labels, pred_labels, target_names=['Human', 'AI'])
print(report)

In [ ]:
model_save_path = './test/ctrl_model'
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

## Models with Adversarial Training

In [ ]:
def adversarial_training(model, inputs, labels, epsilon=0.01):
    model.zero_grad()
    outputs = model(**inputs)
    loss = torch.nn.functional.cross_entropy(outputs.logits, labels)
    loss.backward()  # Compute gradients
    
    # Create perturbed inputs with adversarial noise
    perturbed_inputs = inputs.copy()
    for key in perturbed_inputs:
        if perturbed_inputs[key].requires_grad:
            perturbation = epsilon * perturbed_inputs[key].grad.sign()
            perturbed_inputs[key] = perturbed_inputs[key] + perturbation.detach()  # detach to avoid further graph tracking

    # Clear past gradients
    model.zero_grad()

    # Re-run the model on perturbed data
    perturbed_outputs = model(**perturbed_inputs)
    perturbed_loss = torch.nn.functional.cross_entropy(perturbed_outputs.logits, labels)

    return loss.item(), perturbed_loss.item()

In [ ]:
# Load Data
class TextDataset(Dataset):
    def __init__(self, tokenizer, filepath):
        self.data = pd.read_csv(filepath)
        self.tokenizer = tokenizer
        self.texts = self.data['review'].tolist()
        # Convert labels to integers
        self.labels = torch.tensor(self.data['label'].apply(lambda x: 0 if x == 'human' else 1).tolist())

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        inputs = self.tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
        input_ids = inputs['input_ids'].squeeze(0)  # Remove batch dimension
        return input_ids, label

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()
    true_labels, predictions = [], []

    with torch.no_grad():
        for batch_inputs, batch_labels in dataloader:
            inputs = {'input_ids': batch_inputs}
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_labels = torch.argmax(logits, dim=1)
            predictions.extend(predicted_labels.numpy())
            true_labels.extend(batch_labels.numpy())

    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')
    conf_matrix = confusion_matrix(true_labels, predictions)

    return accuracy, precision, recall, f1, conf_matrix

### BERT

In [ ]:
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load data
dataset = TextDataset(tokenizer, 'reviews.csv')

# Split the dataset
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

# Create data loaders
train_loader = DataLoader(train, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

In [ ]:
# Initialize model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Set model to training mode
model.train()

for epoch in range(2):  # Loop over the dataset multiple times
    total_loss = 0
    total_adv_loss = 0
    for batch_inputs, batch_labels in train_loader:
        inputs = {'input_ids': batch_inputs, 'labels': batch_labels}
        loss, adv_loss = adversarial_training(model, inputs, batch_labels, epsilon=0.01)
        optimizer.step()
        total_loss += loss
        total_adv_loss += adv_loss

    avg_loss = total_loss / len(train_loader)
    avg_adv_loss = total_adv_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Loss: {avg_loss}, Adversarial Loss: {avg_adv_loss}')

In [ ]:
accuracy, precision, recall, f1, conf_matrix = evaluate_model(model, test_loader)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')

# Display confusion matrix for the test set
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix for Test Set')
plt.show()

In [ ]:
output_dir = './test/bert_adv_model'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f'Model saved to {output_dir}')

### RoBERTa

In [ ]:
# Initialize tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Load data
dataset = TextDataset(tokenizer, 'reviews.csv')

# Split the dataset
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

# Create data loaders
train_loader = DataLoader(train, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

In [ ]:
# Initialize model and tokenizer
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Set model to training mode
model.train()

for epoch in range(2):  # Loop over the dataset multiple times
    total_loss = 0
    total_adv_loss = 0
    for batch_inputs, batch_labels in train_loader:
        inputs = {'input_ids': batch_inputs, 'labels': batch_labels}
        loss, adv_loss = adversarial_training(model, inputs, batch_labels, epsilon=0.01)
        optimizer.step()
        total_loss += loss
        total_adv_loss += adv_loss

    avg_loss = total_loss / len(train_loader)
    avg_adv_loss = total_adv_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Loss: {avg_loss}, Adversarial Loss: {avg_adv_loss}')

In [ ]:
accuracy, precision, recall, f1, conf_matrix = evaluate_model(model, test_loader)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')

# Display confusion matrix for the test set
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix for Test Set')
plt.show()

In [ ]:
output_dir = './test/roberta_adv_model'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f'Model saved to {output_dir}')

### GPT-2

In [ ]:
# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load data
dataset = TextDataset(tokenizer, 'reviews.csv')

# Split the dataset
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

# Create data loaders
train_loader = DataLoader(train, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

In [ ]:
# Initialize model and tokenizer
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Set model to training mode
model.train()

for epoch in range(2):  # Loop over the dataset multiple times
    total_loss = 0
    total_adv_loss = 0
    for batch_inputs, batch_labels in train_loader:
        inputs = {'input_ids': batch_inputs, 'labels': batch_labels}
        loss, adv_loss = adversarial_training(model, inputs, batch_labels, epsilon=0.01)
        optimizer.step()
        total_loss += loss
        total_adv_loss += adv_loss

    avg_loss = total_loss / len(train_loader)
    avg_adv_loss = total_adv_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Loss: {avg_loss}, Adversarial Loss: {avg_adv_loss}')

In [ ]:
accuracy, precision, recall, f1, conf_matrix = evaluate_model(model, test_loader)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')

# Display confusion matrix for the test set
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix for Test Set')
plt.show()

In [ ]:
output_dir = './test/gpt2_adv_model'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f'Model saved to {output_dir}')

### XLNet

In [ ]:
# Initialize tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Load data
dataset = TextDataset(tokenizer, 'reviews.csv')

# Split the dataset
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

# Create data loaders
train_loader = DataLoader(train, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

In [ ]:
# Initialize model and tokenizer
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Set model to training mode
model.train()

for epoch in range(2):  # Loop over the dataset multiple times
    total_loss = 0
    total_adv_loss = 0
    for batch_inputs, batch_labels in train_loader:
        inputs = {'input_ids': batch_inputs, 'labels': batch_labels}
        loss, adv_loss = adversarial_training(model, inputs, batch_labels, epsilon=0.01)
        optimizer.step()
        total_loss += loss
        total_adv_loss += adv_loss

    avg_loss = total_loss / len(train_loader)
    avg_adv_loss = total_adv_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Loss: {avg_loss}, Adversarial Loss: {avg_adv_loss}')

In [ ]:
accuracy, precision, recall, f1, conf_matrix = evaluate_model(model, test_loader)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')

# Display confusion matrix for the test set
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix for Test Set')
plt.show()

In [ ]:
output_dir = './test/xlnet_adv_model'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f'Model saved to {output_dir}')

### Grover

In [ ]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('grover-base')

# Load data
dataset = TextDataset(tokenizer, 'reviews.csv')

# Split the dataset
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

# Create data loaders
train_loader = DataLoader(train, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

In [ ]:
# Initialize model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('grover-base', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Set model to training mode
model.train()

for epoch in range(2):  # Loop over the dataset multiple times
    total_loss = 0
    total_adv_loss = 0
    for batch_inputs, batch_labels in train_loader:
        inputs = {'input_ids': batch_inputs, 'labels': batch_labels}
        loss, adv_loss = adversarial_training(model, inputs, batch_labels, epsilon=0.01)
        optimizer.step()
        total_loss += loss
        total_adv_loss += adv_loss

    avg_loss = total_loss / len(train_loader)
    avg_adv_loss = total_adv_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Loss: {avg_loss}, Adversarial Loss: {avg_adv_loss}')

In [ ]:
accuracy, precision, recall, f1, conf_matrix = evaluate_model(model, test_loader)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')

# Display confusion matrix for the test set
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix for Test Set')
plt.show()

In [ ]:
output_dir = './test/grover_adv_model'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f'Model saved to {output_dir}')

### XLM

In [ ]:
# Initialize tokenizer
tokenizer = XLMTokenizer.from_pretrained('xlm-mlm-en-2048')

# Load data
dataset = TextDataset(tokenizer, 'reviews.csv')

# Split the dataset
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

# Create data loaders
train_loader = DataLoader(train, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

In [ ]:
# Initialize model and tokenizer
model = XLMForSequenceClassification.from_pretrained('xlm-mlm-en-2048', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Set model to training mode
model.train()

for epoch in range(2):  # Loop over the dataset multiple times
    total_loss = 0
    total_adv_loss = 0
    for batch_inputs, batch_labels in train_loader:
        inputs = {'input_ids': batch_inputs, 'labels': batch_labels}
        loss, adv_loss = adversarial_training(model, inputs, batch_labels, epsilon=0.01)
        optimizer.step()
        total_loss += loss
        total_adv_loss += adv_loss

    avg_loss = total_loss / len(train_loader)
    avg_adv_loss = total_adv_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Loss: {avg_loss}, Adversarial Loss: {avg_adv_loss}')

In [ ]:
accuracy, precision, recall, f1, conf_matrix = evaluate_model(model, test_loader)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')

# Display confusion matrix for the test set
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix for Test Set')
plt.show()

In [ ]:
output_dir = './test/xlm_adv_model'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f'Model saved to {output_dir}')

### CTRL

In [ ]:
# Initialize tokenizer
tokenizer = CTRLTokenizer.from_pretrained('ctrl')

# Load data
dataset = TextDataset(tokenizer, 'reviews.csv')

# Split the dataset
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

# Create data loaders
train_loader = DataLoader(train, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

In [ ]:
# Initialize model and tokenizer
model = CTRLForSequenceClassification.from_pretrained('ctrl', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Set model to training mode
model.train()

for epoch in range(2):  # Loop over the dataset multiple times
    total_loss = 0
    total_adv_loss = 0
    for batch_inputs, batch_labels in train_loader:
        inputs = {'input_ids': batch_inputs, 'labels': batch_labels}
        loss, adv_loss = adversarial_training(model, inputs, batch_labels, epsilon=0.01)
        optimizer.step()
        total_loss += loss
        total_adv_loss += adv_loss

    avg_loss = total_loss / len(train_loader)
    avg_adv_loss = total_adv_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Loss: {avg_loss}, Adversarial Loss: {avg_adv_loss}')

In [ ]:
accuracy, precision, recall, f1, conf_matrix = evaluate_model(model, test_loader)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')

# Display confusion matrix for the test set
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['AI', 'Human'], yticklabels=['AI', 'Human'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix for Test Set')
plt.show()

In [ ]:
output_dir = './test/ctrl_adv_model'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f'Model saved to {output_dir}')